In [1]:
import torch
from opendataval.dataloader import DataFetcher
from opendataval.experiment import save_dataval, ExperimentMediator

from src.baseline.MLP_hyperparameters import CLASSIFIER_MLP_HYPERPARAMETERS, CLASSIFIER_LEARNING_RATE, \
    CLASSIFIER_BATCH_SIZE
from src.LossVal.LossVal_MLP import LossVal_MLP
from src.LossVal.LossVal_evaluator import LossVal_Evaluator

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [2]:
data_fetcher = DataFetcher("challenge-iris", cache_dir="./../data_files/")
LossVal_evaluator = LossVal_Evaluator(device=torch.device("cuda"), nr_epochs=30, loss_function="Weighted CE loss with squared Sinkhorn")

# Simplify the name for the leaderboard:
LossVal_evaluator.__repr__ = lambda: "LossVal"


In [3]:
NR_LAYERS = CLASSIFIER_MLP_HYPERPARAMETERS["layers"]
HIDDEN_DIM = CLASSIFIER_MLP_HYPERPARAMETERS["hidden_dim"]
ACTIVATION_FN = torch.nn.ReLU()

TRAIN_ARGS = {
    "batch_size": CLASSIFIER_BATCH_SIZE,
    "lr": CLASSIFIER_LEARNING_RATE,
    "epochs": 30
}

In [4]:
# Prepare the model and the experiment:
model = LossVal_MLP(
    is_classification=True,
    layers=NR_LAYERS,
    hidden_dim=HIDDEN_DIM,
    act_fn=ACTIVATION_FN,
    input_dim=data_fetcher.covar_dim[0],
    output_dim=data_fetcher.label_dim[0],
    training_set_size=len(data_fetcher.x_train)
)

exper_med = ExperimentMediator(data_fetcher, model, metric_name="accuracy", train_kwargs=TRAIN_ARGS, output_dir="./../results/iris_challenge/")
evaluators = [LossVal_evaluator]

In [5]:
result_df = exper_med.compute_data_values(evaluators).evaluate(save_dataval, save_output=True)
result_df

Elapsed time LossVal: 0:00:00.700959


,indices,data_values
LossVal,0,-1.471044
LossVal,1,-1.542577
LossVal,2,-1.200364
LossVal,3,-1.471352
LossVal,4,-1.200375
...,...,...
LossVal,100,-1.542452
LossVal,101,-1.5426
LossVal,102,-1.471227
LossVal,103,-1.200361
